In [171]:
import psycopg2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

def connect_to_nfl_db():
    conn = psycopg2.connect("dbname=nfldb user=nfldb")
    return conn


def get_season_game_yards(season, conn):
    season_data = pd.read_sql("""
                select plays.gsis_id, drive.pos_team, games.season_year,
                    case when drive.pos_team = games.home_team then 1 else 0 end as home_team,
                    sum(plays.passing_yds) as passing_yds
                from agg_play plays
                left join game games on plays.gsis_id = games.gsis_id
                left join drive drive on plays.gsis_id = drive.gsis_id
                    and plays.drive_id = drive.drive_id
                where games.season_year={0}
                group by plays.gsis_id, drive.pos_team, games.season_year, games.home_team
                """.format(season), conn)
    return season_data


def combine_rows(df):
    away = df[df['home_team'] == 0]
    away = away.rename(columns={'pos_team' : 'away_team',
                           'passing_yds' : 'away_passing_yds'})
    del away['home_team']
    home = df[df['home_team'] == 1]
    del home['home_team']
    home = home.rename(columns={'pos_team' : 'home_team',
                           'passing_yds' : 'home_passing_yds'})
    return away.merge(home, how='left')


def add_info(info, game):
    info['home_winner'] = np.where(info['home_score'] > info['away_score'], 1, 0)
    info_small = info[['gsis_id', 'home_winner']]
    merge = game.merge(info_small, how="left")
    merge["win_pass_yd_diff"] = np.where(merge['home_winner'] == 1,
                                         merge['home_passing_yds'] - merge['away_passing_yds'],
                                         merge['away_passing_yds'] - merge['home_passing_yds'])
    return merge
    
    
def get_season_game_info(season, conn):
    season_info = pd.read_sql("""
                select * from game where season_year = {0}""".format(season), conn)
    return season_info

In [169]:
conn = connect_to_nfl_db()
year = "2014"
data_2014 = get_season_game_yards(year, conn)
info_2014 = get_season_game_info(year, conn)
combined_2014 = combine_rows(data_2014)
diff_2014 = add_info(info_2014, combined_2014)

In [173]:
diff_2014['win_pass_yd_diff'].describe()

count    332.000000
mean      20.135542
std       92.670597
min     -211.000000
25%      -44.500000
50%       21.500000
75%       84.000000
max      280.000000
Name: win_pass_yd_diff, dtype: float64

In [159]:
combined_2014

,gsis_id,away_team,season_year,away_passing_yds,home_team,home_passing_yds
0,2014080300,NYG,2014,208,BUF,180
1,2014080700,IND,2014,183,NYJ,146
2,2014080701,SF,2014,126,BAL,168
3,2014080702,NE,2014,212,WAS,214
4,2014080703,CIN,2014,209,KC,182
5,2014080704,SEA,2014,221,DEN,163
6,2014080705,DAL,2014,204,SD,243
7,2014080800,MIA,2014,186,ATL,288
8,2014080801,BUF,2014,183,CAR,258
9,2014080802,TB,2014,174,JAC,180


In [118]:
away.head()

,gsis_id,away_team,season_year,away_passing_yds
1,2014080300,NYG,2014,208
2,2014080700,IND,2014,183
5,2014080701,SF,2014,126
6,2014080702,NE,2014,212
8,2014080703,CIN,2014,209


In [120]:
away.merge(home, how='left')

,gsis_id,away_team,season_year,away_passing_yds,home_team,home_passing_yds
0,2014080300,NYG,2014,208,BUF,180
1,2014080700,IND,2014,183,NYJ,146
2,2014080701,SF,2014,126,BAL,168
3,2014080702,NE,2014,212,WAS,214
4,2014080703,CIN,2014,209,KC,182
5,2014080704,SEA,2014,221,DEN,163
6,2014080705,DAL,2014,204,SD,243
7,2014080800,MIA,2014,186,ATL,288
8,2014080801,BUF,2014,183,CAR,258
9,2014080802,TB,2014,174,JAC,180
